In [9]:
USE GraphDemo
GO


Commands completed successfully.

Total execution time: 00:00:00

Domyśnie tabele grafowe nie nakładają ograniczeń na tabele dla źródła i celu krawędzi, co może prowadzić do błędów modelu dancyh.

Aby ograniczyć schemat naszej bazy grafowej możemy nałożyć ograniczenia (`CONSTRAINT`) typu `CONNECTION`. Ograniczenie takie możemy nałożyć na tabele krawędziowe, aby określić źródło i cel krawędzi.

Przy utworzeniu ograniczenia musimy wybrać akcję przy usunięciu węzła. 
- Krawędź wiążąca dany węzeł może zostać kaskadowo usunięta
- Usunięcie węzła może zostać udaremnione i spowodować błąd

Ograniczenia możemy nakładać przy okazji tworzenia nowych tabel:

In [5]:
CREATE TABLE visited
(
   VisitDate INT
      ,CONSTRAINT EC_VISITED CONNECTION (Person TO Restaurant) ON DELETE NO ACTION
)
AS EDGE;


: Msg 13931, Level 16, State 1, Line 1
Edge constraint 'EC_VISITED' references invalid table 'Restaurant'.  Table could not be found.

: Msg 1750, Level 16, State 1, Line 1
Could not create constraint or index. See previous errors.

Total execution time: 00:00:00.021

lub dodawać do istniejących:

In [6]:
ALTER TABLE friendOf ADD CONSTRAINT EC_FRIEND CONNECTION (Person TO Person);


: Msg 4902, Level 16, State 1, Line 1
Cannot find the object "friendOf" because it does not exist or you do not have permissions.

Total execution time: 00:00:00.001

Jeśli chcemy dodać ograniczenie, które zezwala na skierowanie krawędzi z/do więcej niż jednej tabeli możemy użyć ograniczenia wielokrotnego.

**Uwaga!** Jedno ograniczenie wielokrotne to nie to samo co kilka osobnych ograniczeń.

Przykładowo poniższa kwerenda wymusi koniunkcję ograniczeń, a w rezultacie uniemożliwi dodanie nowych krawędzi tego typu

In [14]:
ALTER TABLE owesMoney ADD CONSTRAINT EC_OWES_B CONNECTION (Person TO Bank)
ALTER TABLE owesMoney ADD CONSTRAINT EC_OWES_P CONNECTION (Person TO Person)


Commands completed successfully.

Total execution time: 00:00:00.023

In [15]:
INSERT INTO owesMoney VALUEs
(
(SELECT $node_id FROM Person WHERE Person.name = 'Jacob'),
(SELECT $node_id FROM Bank WHERE Bank.name = 'WAW Bank'),
150
)

INSERT INTO owesMoney VALUEs
(
(SELECT $node_id FROM Person WHERE Person.name = 'Jacob'),
(SELECT $node_id FROM Person WHERE Person.name = 'Julie'),
150
)

: Msg 547, Level 16, State 0, Line 1
The INSERT statement conflicted with the EDGE constraint "EC_OWES_P". The conflict occurred in database "GraphDemo", table "dbo.owesMoney".

The statement has been terminated.

: Msg 547, Level 16, State 0, Line 8
The INSERT statement conflicted with the EDGE constraint "EC_OWES_B". The conflict occurred in database "GraphDemo", table "dbo.owesMoney".

The statement has been terminated.

Total execution time: 00:00:00.030

SQL Server nie daje możliwości modyfikacji istniejących ograniczeń typu CONNECTION

Usuńmy zatem te ograniczenia i zastąpmy je nowym - takim które dopuszcza skierowanie krawędzi do dowolnej z tabel:

In [16]:
ALTER TABLE owesMoney DROP CONSTRAINT EC_OWES_B
ALTER TABLE owesMoney DROP CONSTRAINT EC_OWES_P

ALTER TABLE owesMoney ADD CONSTRAINT EC_OWES CONNECTION (Person TO Bank, Person TO Person) ON DELETE NO ACTION -- You can't just doge your debt by removing yourself from the database


: Msg 2714, Level 16, State 5, Line 4
There is already an object named 'EC_OWES' in the database.

: Msg 1750, Level 16, State 1, Line 4
Could not create constraint or index. See previous errors.

Total execution time: 00:00:00.008

Listę bieżących ograniczeń możemy wyświetlić za pomocą kwerendy

In [17]:
SELECT
   EC.name AS edge_constraint_name
   , OBJECT_NAME(EC.parent_object_id) AS edge_table_name
   , OBJECT_NAME(ECC.from_object_id) AS from_node_table_name
   , OBJECT_NAME(ECC.to_object_id) AS to_node_table_name
   , is_disabled
   , is_not_trusted
FROM sys.edge_constraints EC
   INNER JOIN sys.edge_constraint_clauses ECC
   ON EC.object_id = ECC.object_id
WHERE EC.parent_object_id = object_id('owesMoney');


(2 rows affected)

Total execution time: 00:00:00.028

edge_constraint_name,edge_table_name,from_node_table_name,to_node_table_name,is_disabled,is_not_trusted
EC_OWES,owesMoney,Person,Bank,0,0
EC_OWES,owesMoney,Person,Person,0,0
